In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

# 本地因子库

## HDF5DB

继承自 WritableFactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "HDF5DBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "HDF5DB"
* 主目录: str, 无默认值必须指定, 存放数据的主目录
* 锁目录: str, 存放锁文件的目录, 默认和主目录相同
* 文件打开重试次数: float(可取 inf), 打开数据文件错误时的重试次数

![HDF5 因子库](../images/HDF5因子库.png)

主目录下的每个文件夹表示一张因子表, 每个文件夹下的扩展名为 hdf5 的文件存储了一个因子的数据。每个因子数据的 [HDF5 文件](https://www.hdfgroup.org/) 的 root group 下有三个 dataset:
* ID: 存储因子的 ID 序列数据, shape=(None,), dtype=String, 编码为 utf-8。
* DateTime: 存储因子的时点序列数据, shape=(None,), dtype=float, 时点转换成 timestamp 存储。
* Data: 存储因子数据, shape=(None, None), 行数等于 DateTime 的长度, 列数等于 ID 的长度。double 类型的因子数据存储为 float64 类型，string 类型的因子数据存储为 String 类型(编码为 utf-8)，object 类型的因子数据存储为 vlen_dtype(np.uint8) 类型。

因子数据 HDF5 文件 root group 的 attribute 里存储了因子的元信息。因子表的元信息存储在表目录下的 `_TableInfo.h5` 文件(没有该文件说明还未写入过元信息)中。

锁目录下的 "LockFile" 文件为库锁，修改因子库、因子表以及因子相关信息（比如创建、重命名、删除等操作）时需要获取库锁。锁目录下每个文件夹代表一张因子表，每个文件夹里的 "LockFile" 文件为表锁，读写因子数据时需要获取表锁。

In [2]:
# HDF5DB
# HDF5DB = QS.FactorDB.HDF5DB(sys_args={"主目录": "../Data/HDF5"}).connect()
HDF5DB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect()

print("参数集 : ", HDF5DB.Args)

参数集 :  {'名称': 'Demo库', '主目录': '../Data/HDF5', '锁目录': '', '文件打开重试次数': inf}


### 因子表

因子表参数集:
* 回溯天数: float(可取 inf), 默认值 0, 缺失填充回溯的天数
* 只起始日回溯: bool, 默认值 False, 如果为 True, 表示只对提取数据的第一个时点进行缺失填充, 之后的时点不填充
* 只回溯非目标日: bool, 默认值 False, 如果为 True, 表示只用不在提取时点序列中的数据进行缺失填充
* 只回溯时点: bool, 默认值 False, 如果为 True, 表示所有因子统一沿着时点字段进行回溯填充, 不单独填充

In [3]:
# HDF5DB 因子表
print("因子表 : ", HDF5DB.TableNames)
print("===============================================")

Args = {
    "回溯天数": 4,
    "只起始日回溯": False,
    "只回溯非目标日": False,
    "只回溯时点": False,
}

DTs = [dt.datetime(2019,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

FT = HDF5DB.getTable("stock_cn_day_bar_nafilled", args=Args)
print(f"因子表 '{FT.Name}' 的因子 : ", FT.FactorNames)
print("===============================================")
print("参数 : ")
print(Args)
print("数据 : ")
print(FT.readData(factor_names=["open", "close"], ids=IDs, dts=DTs).iloc[:, :, 0])

因子表 :  ['stock_cn_day_bar_nafilled', 'stock_cn_index_component']
因子表 'stock_cn_day_bar_nafilled' 的因子 :  ['close', 'open']
参数 : 
{'回溯天数': 4, '只起始日回溯': False, '只回溯非目标日': False, '只回溯时点': False}
数据 : 
            open  close
2019-01-01  9.31   9.38
2019-01-02  9.39   9.19
2019-01-03  9.18   9.28
2019-01-04  9.24   9.75
2019-01-05  9.24   9.75
2019-01-06  9.24   9.75
2019-01-07  9.84   9.74
2019-01-08  9.73   9.66
2019-01-09  9.74   9.94


## ZarrDB

继承自 WritableFactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "ZarrDBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "ZarrDB"
* 主目录: str, 无默认值必须指定, 存放数据的主目录

![Zarr 因子库](../images/Zarr因子库.png)

主目录下的每个 [zarr 文件](https://zarr.readthedocs.io/en/stable/) 表示一张因子表, 每个文件的 root group 下的 group 代表一个因子，每个因子 group 下有三个 dataset:
* ID: 存储因子的 ID 序列数据
* DateTime: 存储因子的时点序列数据
* Data: 存储因子数据, 行数等于 DateTime 的长度, 列数等于 ID 的长度。

因子 group 的 attribute 里存储了因子的元信息。因子表的元信息存储在 zarr 文件 root group 的 attribute 中。

目前的实现下数据主目录和锁目录没有分离，主目录下的 "LockFile" 文件为库锁，没有表锁，所有操作均需获取库锁。

In [21]:
# # 生成Demo数据
# import QuantStudio.FactorDataBase.ZarrDB
# TDB = QuantStudio.FactorDataBase.ZarrDB.ZarrDB(sys_args={"主目录": r"..\Data\Zarr"})
# TDB.connect();

# SDB = QS.FactorDB.HDF5DB(sys_args={"主目录": r"..\Data\HDF5"})
# SDB.connect();

# # 生成Demo数据：stock_cn_quote_adj_no_nafilled
# FT = SDB.getTable("stock_cn_quote_adj_no_nafilled")
# IDs = FT.getID()[:50]
# DTs = FT.getDateTime()[-120:]
# SData = FT.readData(factor_names=FT.FactorNames, ids=IDs, dts=DTs)
# TDB.writeData(data=SData, table_name="stock_cn_quote_adj_no_nafilled", if_exists="update")

# # 生成Demo数据：stock_cn_index_component
# FT = SDB.getTable("stock_cn_index_component")
# IDs = FT.getID()
# DTs = FT.getDateTime()[-50:]
# SData = FT.readData(factor_names=FT.FactorNames, ids=IDs, dts=DTs)
# TDB.writeData(data=SData, table_name="stock_cn_index_component", if_exists="update")

# # 删除多余的表
# for iTableName in TDB.TableNames:
#     if iTableName not in ['stock_cn_index_component', 'stock_cn_quote_adj_no_nafilled']:
#         TDB.deleteTable(iTableName)

In [4]:
# ZarrDB
import QSExt.FactorDataBase.ZarrDB
ZarrDB = QSExt.FactorDataBase.ZarrDB.ZarrDB(sys_args={"主目录": "../Data/Zarr"})
ZarrDB.connect();

print("参数集 : ", ZarrDB.Args)

参数集 :  {'名称': 'ZarrDB', '主目录': '../Data/Zarr'}


### 因子表

In [5]:
# ZarrDB 因子表
print("因子表 : ", ZarrDB.TableNames)
print("===============================================")

Args = {}

DTs = [dt.datetime(2019,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

FT = ZarrDB.getTable("stock_cn_day_bar_adj_no_nafilled", args=Args)
print(f"因子表 '{FT.Name}' 的因子 : ", FT.FactorNames)
print("===============================================")
print(FT.readData(factor_names=["high", "close"], ids=IDs, dts=DTs).iloc[:, :, 0])

因子表 :  ['stock_cn_day_bar_adj_no_nafilled', 'stock_cn_index_component']
因子表 'stock_cn_day_bar_adj_no_nafilled' 的因子 :  ['amount', 'chg', 'close', 'high', 'if_trading', 'low', 'negotiable_market_cap', 'open', 'pre_close', 'total_market_cap', 'turnover_rate', 'volume']
             high  close
2019-01-01    NaN    NaN
2019-01-02   9.42   9.19
2019-01-03   9.33   9.28
2019-01-04   9.82   9.75
2019-01-05    NaN    NaN
2019-01-06    NaN    NaN
2019-01-07   9.85   9.74
2019-01-08   9.74   9.66
2019-01-09  10.08   9.94


## CSVDB(TODO)

继承自 WritableFactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "CSVDBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "CSVDB"
* 主目录: str, 无默认值必须指定, 存放数据的主目录
* 字符编码: str, 默认值 "utf-8", csv 文件的编码格式

![CSV 因子库](../images/CSV因子库.png)

主目录下的每个文件夹表示一张因子表, 每个文件夹下的 csv 文件存储了一个因子的数据。每个 csv 文件的第一行第一列记录了因子的 DataType(TODO)，第一行表示因子的 ID 序列，第一列表示因子的时点序列，其余为 Data。

目前的实现下数据主目录和锁目录没有分离，主目录下的 "LockFile" 文件为库锁，修改因子库、因子表以及因子相关信息（比如创建、重命名、删除等操作）时需要获取库锁。主目录下每个因子表文件夹里的 "LockFile" 文件为表锁，读写因子数据时需要获取表锁。

In [2]:
# # 生成Demo数据
# import QuantStudio.FactorDataBase.CSVDB
# TDB = QuantStudio.FactorDataBase.CSVDB.CSVDB(sys_args={"主目录": r"..\Data\CSV"})
# TDB.connect();

# SDB = QS.FactorDB.HDF5DB(sys_args={"主目录": r"..\Data\HDF5"})
# SDB.connect();

# # 生成Demo数据：stock_cn_quote_adj_no_nafilled
# FT = SDB.getTable("stock_cn_quote_adj_no_nafilled")
# IDs = FT.getID()[:50]
# DTs = FT.getDateTime()[-120:]
# SData = FT.readData(factor_names=FT.FactorNames, ids=IDs, dts=DTs)
# TDB.writeData(data=SData, table_name="stock_cn_quote_adj_no_nafilled", if_exists="update")

# # 生成Demo数据：stock_cn_index_component
# FT = SDB.getTable("stock_cn_index_component")
# IDs = FT.getID()
# DTs = FT.getDateTime()[-50:]
# SData = FT.readData(factor_names=FT.FactorNames, ids=IDs, dts=DTs)
# TDB.writeData(data=SData, table_name="stock_cn_index_component", if_exists="update")

# # 删除多余的表
# for iTableName in TDB.TableNames:
#     if iTableName not in ['stock_cn_index_component', 'stock_cn_quote_adj_no_nafilled']:
#         TDB.deleteTable(iTableName)

In [2]:
# CSVDB
import QSExt.FactorDataBase.CSVDB
CSVDB = QSExt.FactorDataBase.CSVDB.CSVDB(sys_args={"主目录": "../Data/CSV"})
CSVDB.connect();

print("参数集 : ", CSVDB.Args)

参数集 :  {'名称': 'CSVDB', '主目录': '../Data/CSV', '字符编码': 'utf-8'}


### 因子表

In [3]:
# CSVDB 因子表
print("因子表 : ", CSVDB.TableNames)
print("===============================================")

Args = {}

DTs = [dt.datetime(2019,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

FT = CSVDB.getTable("stock_cn_day_bar_adj_no_nafilled", args=Args)
print(f"因子表 '{FT.Name}' 的因子 : ", FT.FactorNames)
print("===============================================")
print(FT.readData(factor_names=["high", "close"], ids=IDs, dts=DTs).iloc[:, :, 0])

因子表 :  ['stock_cn_day_bar_adj_no_nafilled', 'stock_cn_index_component']
因子表 'stock_cn_day_bar_adj_no_nafilled' 的因子 :  ['amount', 'chg', 'close', 'high', 'if_trading', 'low', 'negotiable_market_cap', 'open', 'pre_close', 'total_market_cap', 'turnover_rate', 'volume']
             high  close
2019-01-01    NaN    NaN
2019-01-02   9.42   9.19
2019-01-03   9.33   9.28
2019-01-04   9.82   9.75
2019-01-05    NaN    NaN
2019-01-06    NaN    NaN
2019-01-07   9.85   9.74
2019-01-08   9.74   9.66
2019-01-09  10.08   9.94
